## Prticle swarm algorithm

In [1]:
import numpy as np
from itertools import cycle
import xml.etree.ElementTree as ET
MIN_DURATION,MAX_DURATION = 5 , 50
CYCLE_LENGTH = 4
class particule():
    def __init__(self,d):
        self.pos = np.random.uniform(MIN_DURATION,MAX_DURATION,CYCLE_LENGTH)
        self.Pbest = self.pos
        self.Pbestcost = float('inf')
        self.d = d
        self.speed =np.random.uniform(-12,-12, CYCLE_LENGTH)

    def setPos(self,):
        
        self.pos = np.clip(self.speed + self.pos,MIN_DURATION,MAX_DURATION)

    def setSpeed(self,w,alpha,beta,Gbest):
        self.speed = np.clip(w* self.speed + alpha * np.random.rand()*(self.Pbest - self.pos) + beta * np.random.rand()*(Gbest - self.pos)\
                             ,-5,5)
        
    def setPbest(self,score):
        self.Pbest = self.pos
        self.Pbestcost = score
        


## single intersection simulation

In [2]:

import os
import sys
import optparse
import random

# we need to import python modules from the $SUMO_HOME/tools directory
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

from sumolib import checkBinary  # noqa
import traci  # noqa



import traci.constants as tc
first = True
def run(particle):
    """execute the TraCI control loop"""
    step = 0
    
    network = ET.parse('data/cross.net.xml')
    signal = network.find('tlLogic') 
    dur = [34,6,31,6] 
    i = 0
    global first
    for phase in signal.iter('phase'):
        #print('phase')
        if first:
            phase.set("duration", str(dur[i]))
        else:
            phase.set("duration", str(int(particle.pos[i])))
        i+=1
        first = False

    network.write("data/cross.net.xml")

    #
    test = cycle([0,1,2,3])
    LastPhase = 0
    s = 0
    
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        #test+= 1
        step += 1
        if step >=500:
            break
    #print(s,test)    
    traci.close()
    sys.stdout.flush()
    xmldoc = ET.parse('tripinfo.xml')

    tripinfos = xmldoc.findall('tripinfo')

    waiting_time = 0
    for tripinfo in tripinfos:
        waiting_time += float(tripinfo.get('waitingTime'))
    return waiting_time



# this is the main entry point of this script


In [3]:
#options = get_options()
c = cycle(np.arange(10000))
    # this script has been called from the command line. It will start sumo as a
def function(particle):# server, then connect and run
    sumoBinary = checkBinary('sumo')
       # first, generate the route file for this simulation
    generate_routefile()

        # this is the normal way of using traci. sumo is started as a
        # subprocess and then the python script connects and runs
    traci.start(['sumo', "-c", "data/cross.sumocfg",
                                 "--tripinfo-output", "tripinfo.xml"],port =  8873,label=f"{next(c)}")
    
    s = run(particle)
    return s
       

In [4]:
def pso(pop,function,maxiter = 20,alpha =4, beta = 2 ,w = .9,):
    #initialiser les swarms
    swarms = []
    for i in range(pop):
        #initial_position = np.random.choice([0,1], CYCLE_LENGTH)
        swarms.append(particule(CYCLE_LENGTH))
        #print(swarms[i].pos)
    #initialiser Gbest
    
    costGbest = float('inf')
    Gbest = np.zeros(CYCLE_LENGTH)
    #Gfeatures = float('inf')
    history = []
    while maxiter:
        #a= np.zeros((pop,d+1))
        for i in range(pop):
            swarms[i].setSpeed(w,alpha,beta,Gbest) 
            
            swarms[i].setPos()
            temp_Res = function(swarms[i])
            if swarms[i].Pbestcost > temp_Res:
                
                swarms[i].setPbest(temp_Res)
                if costGbest > swarms[i].Pbestcost:
                    costGbest = swarms[i].Pbestcost
                    Gbest = swarms[i].Pbest
        if maxiter%5 ==0:
            print('best pos  is : ' ,Gbest)
            print('the best time is :',costGbest)
            
          
        maxiter-=1
    return costGbest,Gbest,history

In [5]:
pso(10,function)

best pos  is :  [11.99823124 29.79355629 44.45233948 34.24337188]
the best time is : 995.0
[-5. -5.  5. -5.]
[42.20426821 29.96304722 28.76739722 39.95123594]
1116.0
best pos  is :  [16.49823124 25.29355629 39.95233948 29.74337188]
the best time is : 936.0
[ 5.          2.25490638 -5.          5.        ]
[27.20426821 28.66663838 37.68446985 32.62114617]
1037.0
best pos  is :  [22.99140779 19.63224973 42.08505912 23.23314861]
the best time is : 888.0
[-5.         -0.37297119  5.         -5.        ]
[31.84132095 23.39128866 37.91231286 26.35378563]
1013.0
best pos  is :  [22.99140779 19.63224973 42.08505912 23.23314861]
the best time is : 888.0
[ 5. -5. -5.  5.]
[26.84132095 19.42993493 40.06834558 27.35593066]
928.0


(888.0, array([22.99140779, 19.63224973, 42.08505912, 23.23314861]), [])